In [1]:
# import local modules
# import MongoDB modules
import motor.motor_asyncio
from dotenv import dotenv_values
from openimagingdatamodel.ontology_tools import RadLexConcept, transform_radlex

In [2]:
# establish connection to MongoDB
config = dotenv_values(".env")
client = motor.motor_asyncio.AsyncIOMotorClient(config["ATLAS_DSN"])
db = client["ontologies"]
collection = db["RadLex"]
# Get the count of documents in the collection to confirm that the data was loaded
count = await collection.count_documents({})
print(f"Count of documents in the collection: {count}")

Count of documents in the collection: 46761


In [3]:
# Access the ontologies database and "RadLex" collection
doc = await collection.find_one()

In [4]:
print(doc)

{'_id': ObjectId('65f84ed1f80fad5323c79c1f'), 'Class ID': 'http://radlex.org/RID/RID35591', 'Preferred Label': 'string-of-pearls sign of bowel', 'Definitions': 'Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.', 'Obsolete': False, 'Parents': 'http://radlex.org/RID/RID29023', 'http://data': {'bioontology': {'org/metadata/prefixIRI': 'RID35591'}}, 'http://radlex': {'org/RID/Anatomical_Site': 'http://radlex.org/RID/RID132', 'org/RID/Comment': 'http://radiology.rsna.org/cgi/content/full/214/1/157', 'org/RID/Definition': 'Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.', 'org/RID/May_Be_Caused_By': 'http://radlex.org/RID/RID4962', 'org/RID/Preferred_name': 'string-of-pearls s

### Code to try transformation before writing to MongoDB


In [5]:
t_doc = transform_radlex(doc)
print(t_doc.model_dump_json(indent=2))

{
  "id": "RID35591",
  "preferred_label": "string-of-pearls sign of bowel",
  "synonyms": null,
  "parent": "RID29023",
  "definition": "Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.",
  "radlex_properties": {
    "anatomical_site": "RID132",
    "comment": "157",
    "definition": "Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.",
    "may_be_caused_by": "RID4962",
    "preferred_name": "string-of-pearls sign of bowel",
    "preferred_name_german": "string-of-pearls sign of bowel (EN)",
    "related_modality": "RID10345",
    "source": "Radiology 2000; 214:157-158"
  }
}


In [6]:
# Check multiple documents
docs = await collection.find({}).to_list(length=10)
for doc in docs:
    t_doc = transform_radlex(doc)
    print(t_doc.model_dump_json(indent=2))

{
  "id": "RID35591",
  "preferred_label": "string-of-pearls sign of bowel",
  "synonyms": null,
  "parent": "RID29023",
  "definition": "Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.",
  "radlex_properties": {
    "anatomical_site": "RID132",
    "comment": "157",
    "definition": "Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.",
    "may_be_caused_by": "RID4962",
    "preferred_name": "string-of-pearls sign of bowel",
    "preferred_name_german": "string-of-pearls sign of bowel (EN)",
    "related_modality": "RID10345",
    "source": "Radiology 2000; 214:157-158"
  }
}
{
  "id": "RID45335",
  "preferred_label": "peripheral segment of nerve to left sternocleidomast

# Code to write transformed RadLex to new 'radlex' collection in MongoDB


In [4]:
# Create a list to store the transformed documents
transformed_docs = []


# iterate over documents in "RadLex" collection
#for doc in radlex_old.find():
    # Transform the document
    transformed_doc = Transform(doc)
    # Append the transformed document to the list
    transformed_docs.append(transformed_doc)

NameError: name 'radlex_old' is not defined

In [ ]:
# Access the new "radlex" collection in MongoDB
radlex_new = db["radlex"]


# write the transformed documents to the new "radlex" collection
radlex_new.insert_many(transformed_docs)